In [1]:
import requests
import pandas as pd
import time
from pygame import mixer #for sound
from datetime import datetime, timedelta #for looping over date
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
API_key = 'JTFthLtGYuKFQK47MPvQcrabXJRJPk5nUEgaric4f17oP5VJcHyHqsqCMWXu'

#append to dictipnary 
def app_dictionary(id, list, dictionary):
    if id not in dictionary: #checks if column with key already exists 
        dictionary[id] = [list]
    else:
        dictionary[id].append(list) #if yes, it appends to key value

#running api calls and returning result + pagination
def run_api(api_call):
    try:
        result = requests.get(api_call)
        result = result.json()
    except ValueError:
        print(f"Failed to decode JSON. Please check API call: {api_call}")

    #checks if there is a next page
    #if yes it returns true, if no it returns false
    try:
        has_more = result['pagination']['has_more']
    except:
        has_more = False

    try:
        rate_limit = result['rate_limit']['remaining']
        reset = result['rate_limit']['resets_in_seconds']
    except:
        print('no rate limit available')

    return result, has_more, rate_limit, reset

#function for playing sound
def play_sound():
    mixer.init()
    mixer.music.load('done-for-you.mp3')
    mixer.music.play()

def run_pages(api_first, api_second): 
    results, has_more, rate_limit, reset = run_api(f"{api_first}{api_second}")

    page = 2
    while has_more == True:
        if int(rate_limit) > 1:
            next_results, has_more, rate_limit, reset = run_api(f"{api_first}page={page}&{api_second}")
            results['data'] = results['data'] + next_results['data']
            page += 1
        else:
            print(f'Starting sleep timer for: {reset} seconds...')
            time.sleep(int(reset))
    else:
        pass
        
    return results

### **Load data**
<br>
Need to find which teams have played for which seasons, and then set the time interval to the season duration and only get the data for the correct teams.

In [2]:
teams_df = pd.read_csv('data/teams.csv')
seasons_df = pd.read_csv('data/seasons.csv')
leagues_df = pd.read_csv('data/leagues.csv')

Creating dictionary with seasons start date, end date, id, and teams participating in the season

In [5]:
season_dict = {'id':[], 'starting_at':[], 'ending_at':[], 'name':[]}
for row in seasons_df.index:
    app_dictionary('id', str(seasons_df.iloc[row]['id']), season_dict)
    app_dictionary('starting_at', str(seasons_df.iloc[row]['starting_at']), season_dict)
    app_dictionary('ending_at', seasons_df.iloc[row]['ending_at'], season_dict)
    app_dictionary('teams', [i for i in teams_df.loc[teams_df['season_id'] == int(seasons_df.iloc[row]['id'])]['id']], season_dict)
    app_dictionary('name', str(seasons_df.iloc[row]['name']), season_dict)


In [6]:
season_dict = pd.DataFrame.from_dict(season_dict)
season_dict.to_csv('data/seasons_info.csv')
season_dict

,id,starting_at,ending_at,name,teams
0,19734,2022-08-05,2023-05-28,2022/2023,"[78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 2..."
1,19744,2022-08-05,2023-05-27,2022/2023,"[3317, 1079, 510, 3321, 3543, 68, 82, 3320, 67..."
2,19799,2022-08-12,2023-06-04,2022/2023,"[3477, 3468, 485, 36, 1099, 361, 6827, 645, 23..."
3,19806,2022-08-13,2023-06-11,2022/2023,"[10722, 2930, 397, 522, 1123, 625, 345, 1628, ..."
4,21646,2023-08-11,2024-05-19,2023/2024,"[78, 11, 8, 19, 29, 115, 51, 21, 14, 236, 52, ..."
5,21694,2023-08-11,2024-05-26,2023/2024,"[377, 3477, 3468, 103, 485, 2921, 36, 6827, 64..."
6,21795,2023-08-18,2024-05-18,2023/2024,"[1079, 510, 3321, 3543, 68, 82, 3320, 277, 90,..."
7,21818,2023-08-19,2024-06-02,2023/2024,"[1628, 708, 346, 7743, 113, 102, 109, 43, 37, ..."
8,17361,2020-09-18,2021-05-22,2020/2021,"[3317, 1079, 510, 3321, 3543, 68, 82, 2927, 33..."
9,17420,2020-09-12,2021-05-23,2020/2021,"[78, 11, 8, 19, 42, 29, 71, 51, 21, 14, 20, 13..."


In [8]:
season_name = season_dict['name']
season_name[0].split('/')[0]

'2022'

In [9]:
start_dates = season_dict['starting_at']
end_dates = season_dict['ending_at']
teams_inSeasons = season_dict['teams']
season_id = season_dict['id']
season_name = season_dict['name']

# This allows you to work with all rows at once
for season_start, season_end, teams_inSeason, season_id, season_name in zip(start_dates, end_dates, teams_inSeasons, season_id, season_name):
    print(f'Starting: {season_start}, {season_end}, {teams_inSeason}, {season_id}')
    fixtures = {'fixture_id':[], 'name': [], 'league_id':[], 'season_id':[], 'venue_id':[], 'formation_away':[], 'formation_home': [], 'referee':[]}
    events = []
    weather_pitch = {'fixture_id':[], 'pitch': [], 'weatherDescription':[], 'current':[]}
    name = season_name.split('/')[0]
    for team in teams_inSeason:
        start_date = datetime.strptime(season_start, '%Y-%m-%d')
        end_date = datetime.strptime(season_end, '%Y-%m-%d')
        #find parameters
        total_days = (end_date - start_date).days
        days_interval = 100

        # Variable to store the previous date
        previous_date = None
        # Loop over the 300 days in 100-day intervals\n",
        for i in range(0, total_days, days_interval):
            current_date = start_date + timedelta(days=i)
            formatted_date = current_date.strftime('%Y-%m-%d')

            # Print the previous and current date,
            if previous_date:
                print(f"Previous date: {previous_date} -> Next date: {formatted_date}")
                first = 'https://api.sportmonks.com/v3/football/fixtures/between/' + str(previous_date) + '/' + str(formatted_date) + '/' + str(team) + '?'
                second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567' 
                results = run_pages(first, second)
                
                try: 
                    for result in results['data']:
                        #all events
                        events = events + result['events']

                        #creating weather dataset
                        weather_pitch['fixture_id'].append(result['id'])
                        if result['metadata']:
                            weather_pitch['pitch'].append(result['metadata'][0]['values']['pitch'])
                        else:
                            weather_pitch['pitch'].append('NaN')
                        try:
                            weather_pitch['weatherDescription'].append(result['weatherreport']['description'])
                        except TypeError:
                            weather_pitch['weatherDescription'].append('NaN')
                        try:
                            weather_pitch['current'].append(result['weatherreport']['current']['description'])
                        except TypeError:
                            weather_pitch['current'].append('NaN')

                        #creating fixture dataset
                        fixtures['fixture_id'].append(result['id'])
                        fixtures['name'].append(result['name'])
                        fixtures['league_id'].append(result['league_id'])
                        fixtures['season_id'].append(result['season_id'])
                        fixtures['venue_id'].append(result['venue_id'])
                        if result['formations']:
                            try:
                                fixtures['formation_away'].append(result['formations'][0]['formation'])
                                fixtures['formation_home'].append(result['formations'][1]['formation'])
                            except IndexError:
                                fixtures['formation_away'].append(result['formation'])
                                fixtures['formation_home'].append(result['formation'])
                        else:
                            fixtures['formation_away'].append('NaN')
                            fixtures['formation_home'].append('NaN')
                        #choosing the main referee -> type 6
                        try:
                            fixtures['referee'].append([referee['referee_id'] for referee in result['referees'] if referee['type_id'] == 6][0])
                        except IndexError:
                            fixtures['referee'].append(result['referees'])
                except KeyError:
                    print(f'No data for {team}')
                 
            else: # Avoiding calling only the first date 
                print(f"First date: {formatted_date}") 
        
            previous_date = formatted_date

    #saving dataframe
    events = {'data': events}
    events = pd.DataFrame.from_dict(events['data'])
    fixtures = pd.DataFrame.from_dict(fixtures)
    weather_pitch = pd.DataFrame.from_dict(weather_pitch)

    #dropping duplicates 
    events.drop_duplicates()
    fixtures.drop_duplicates(subset=['fixture_id'], inplace=True)
    weather_pitch.drop_duplicates(subset=['fixture_id'], inplace=True)

    #saving as csv
    events.to_csv(f'data/events/e_{name}_{season_id}.csv')
    fixtures.to_csv(f'data/fixtures/f_{name}_{season_id}.csv') 
    weather_pitch.to_csv(f'data/weather/w_{name}_{season_id}.csv')


Starting: 2022-08-05, 2023-05-28, [78, 11, 8, 19, 42, 29, 71, 51, 14, 236, 52, 20, 63, 13, 9, 1, 65, 18, 15, 6], 19734
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 78
Previous date: 2022-11-13 -> Next date: 2023-02-21
No data for 78
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 11
Previous date: 2022-11-13 -> Next date: 2023-02-21
No data for 11
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 8
Previous date: 2022-11-13 -> Next date: 2023-02-21
No data for 8
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 19
Previous date: 2022-11-13 -> Next date: 2023-02-21
No data for 19
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 42
Previous date: 2022-11-13 -> Next date: 2023-02-21
No data for 42
First date: 2022-08-05
Previous date: 2022-08-05 -> Next date: 2022-11-13
No data for 29
Previous date: 2022

KeyboardInterrupt: 

TEST

In [4]:
first = 'https://api.sportmonks.com/v3/football/fixtures/18165627?'
second = 'api_token=' + API_key + '&per_page=50&select=name&include=metadata;formations;weatherReport:description,current;referees;events:type_id,addition,minute&filters=eventTypes:10,19,20,21;metadataTypes:567'
results = run_pages(first,second)
results

{'data': {'name': 'Real Sociedad vs Rayo Vallecano',
  'id': 18165627,
  'sport_id': 1,
  'round_id': 250017,
  'stage_id': 77454016,
  'group_id': None,
  'aggregate_id': None,
  'league_id': 564,
  'season_id': 18462,
  'venue_id': 133,
  'state_id': 5,
  'starting_at_timestamp': None,
  'metadata': [],
  'formations': [{'id': 152438,
    'fixture_id': 18165627,
    'participant_id': 377,
    'formation': '4-2-3-1',
    'location': 'away'},
   {'id': 152437,
    'fixture_id': 18165627,
    'participant_id': 594,
    'formation': '4-1-4-1',
    'location': 'home'}],
  'weatherreport': None,
  'referees': [{'id': 108451,
    'fixture_id': 18165627,
    'referee_id': 18789,
    'type_id': 6},
   {'id': 2361077, 'fixture_id': 18165627, 'referee_id': 18791, 'type_id': 8},
   {'id': 3530049, 'fixture_id': 18165627, 'referee_id': 43117, 'type_id': 9}],
  'events': [{'id': 3081165,
    'type_id': 19,
    'sub_type_id': 1496,
    'fixture_id': 18165627,
    'player_id': 34053,
    'related_pl

In [176]:
fixtures = {'fixture_id':[], 'name': [], 'league_id':[], 'season_id':[], 'venue_id':[], 'formation_away':[], 'formation_home': [], 'referee':[]}
events = []
weather_pitch = {'fixture_id':[], 'pitch': [], 'weatherDescription':[], 'current':[]}
referee = {}
for result in results['data']:
    #all events
    events = events + result['events']

    #creating weather dataset
    weather_pitch['fixture_id'].append(result['id'])
    if result['metadata']:
        weather_pitch['pitch'].append(result['metadata'][0]['values']['pitch'])
    else:
        weather_pitch['pitch'].append('NaN')
    if result.get('weatherreport') and 'description' in result['weatherreport']:
        weather_pitch['weatherDescription'].append(result['weatherreport']['description'])
    else:
        weather_pitch['weatherDescription'].append('NaN')
    if result.get('weatherreport') and 'description' in result['weatherreport']:
        weather_pitch['current'].append(result['weatherreport']['current'])
    else:
        weather_pitch['current'].append('NaN')
    
    #creating fixture dataset
    fixtures['fixture_id'].append(result['id'])
    fixtures['name'].append(result['name'])
    fixtures['league_id'].append(result['league_id'])
    fixtures['season_id'].append(result['season_id'])
    fixtures['venue_id'].append(result['venue_id'])
    fixtures['formation_away'].append(result['formations'][0]['formation'])
    fixtures['formation_home'].append(result['formations'][1]['formation'])
    #choosing the main referee -> type 6
    fixtures['referee'].append(result['referees'])
